# GRAPES Toolkit — Example 1

This notebook demonstrates a minimal end‑to‑end workflow with **GRAPES**:

1) Load a 3D XCT intensity volume and a corresponding label volume.
2) Visualize representative slices.
3) Build a GRAPES particle table (Pandas DataFrame).
4) Explore radial properties and intensity profiles.
5) Render a simple property map back into image space.

> **Data**: expects `example_image.tif` and `example_labels.tif` in the project root (or run this from the `examples/` folder).


## 0. Environment setup
Make sure we can import local utilities regardless of whether the notebook is run from the repo root or from the `examples/` folder.

In [ ]:
# Standard path setup to allow local imports when launched from repo root or examples/
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "examples":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))

## 1. Imports

In [ ]:
import tifffile as tiff
import matplotlib.pyplot as plt
import pandas as pd
import GRAPES as gp  # core toolkit

# Optional: enable interactive plotting in supported environments
%matplotlib widget  # uncomment in JupyterLab if you want pan/zoom widgets

## 2. Load data
Read the 3D intensity image and its corresponding labeled segmentation volume.

In [ ]:
# File paths (edit if your files live elsewhere)
INTENSITY_PATH = 'examples_data/example_image.tif'
LABELS_PATH = 'examples_data/example_labels.tif'

# Load images (3D)
intensity_image = tiff.imread(INTENSITY_PATH)
labels_image = tiff.imread(LABELS_PATH)

print(f"Intensity shape: {getattr(intensity_image, 'shape', None)}  |  Labels shape: {getattr(labels_image, 'shape', None)}")

## 3. Quick look at a central slice
Plot a representative axial slice from both volumes.

In [ ]:
# Choose a slice index to visualize (middle slice by default)
SLICE_IDX = intensity_image.shape[0] // 2 if hasattr(intensity_image, 'shape') else 80

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(intensity_image[SLICE_IDX], cmap='gray')
plt.title(f'Intensity Image (slice {SLICE_IDX})')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(labels_image[SLICE_IDX], cmap='jet', interpolation='none')
plt.title(f'Labels Image (slice {SLICE_IDX})')
plt.axis('off')

plt.tight_layout()
plt.show()

> The left panel shows the intensity slice of cracked particles from the XCT volume. The right panel shows the segmented labels for the three particles to be analyzed.

## 4. Build the GRAPES particle table
Create a Pandas DataFrame containing per‑particle measurements and radial‑layer statistics.

In [ ]:
# Create the particle table; see help(gp.GRAPES) for available parameters/outputs
particle_df = gp.GRAPES(labels_image, intensity_image)
print(f"Particles detected: {len(particle_df) if hasattr(particle_df, '__len__') else 'unknown'}")

### Peek at the schema

In [ ]:
particle_df.head()

### API reference (concise)
Use `help(gp.GRAPES)` to view function arguments and a summary of outputs.

In [ ]:
help(gp.GRAPES)

## 5. Visualize radial information
Plot a distance‑transform (radial layer) view for a selected particle, then plot radial intensity profiles for all particles.

In [ ]:
# Plot radial layers as an image for a single particle (edit label as needed)
plot = gp.plot_particle_image(particle_df, label=3, image_type='distance_transform')

In [ ]:
# Plot radial intensity profiles for each particle
for i in particle_df['label']:
    gp.plot_radial_intensities(particle_df, i)

> In each particle we observe a drop in grey‑level towards the center of the particle, suggesting higher crack and void formation in central regions.

## 6. Map a property back to image space
Render a per‑particle property (here: `intensity_min`) into a volume aligned with the label image. This produces a quick‑look property map.

In [ ]:
# Create an image where each labeled voxel is assigned the particle's chosen property
min_i_arr = gp.prop_2_image(labels_image, particle_df, prop='intensity_min')

# Visualize the same slice as above
plt.figure(figsize=(5, 5))
plt.imshow(min_i_arr[SLICE_IDX], interpolation='none')
plt.title(f"Particle Property Map: intensity_min (slice {SLICE_IDX})")
plt.axis('off')
plt.tight_layout()
plt.show()

## 7. Next steps
- Try other properties with `prop_2_image`, e.g., `intensity_mean`, `intensity_std`, or custom metrics.
- Use `plot_particle_image(..., image_type='radial_layers')` (if available in your version) to visualize computed layers.
- Save the `particle_df` to CSV for downstream analysis: `particle_df.to_csv('particle_table.csv', index=False)`.
